In [2]:
%cd graph-enhanced-retrieval-qa
!pwd

/home/sslab/24m0786/graph-enhanced-retrieval-qa
/home/sslab/24m0786/graph-enhanced-retrieval-qa


In [3]:
import sys
sys.path.append('..') 

from src.data_loader import load_dataset, process_sample
from rank_bm25 import BM25Okapi
from pprint import pprint

dev_dataset = load_dataset('data/raw/dev.json')

sample = dev_dataset[0]
processed_data = process_sample(sample)
print(processed_data)

print("Question:", processed_data['question'])
print("Ground Truth Titles:", processed_data['ground_truth_titles'])

{'question': 'Who is the mother of the director of film Polish-Russian War (Film)?', 'passages': {'Xawery Żuławski': 'Xawery Żuławski (born 22 December 1971 in Warsaw) is a Polish film director. In 1995 he graduated National Film School in Łódź. He is the son of actress Małgorzata Braunek and director Andrzej Żuławski. His second feature "Wojna polsko-ruska" (2009), adapted from the controversial best-selling novel by Dorota Masłowska, won First Prize in the New Polish Films competition at the 9th Era New Horizons Film Festival in Wrocław. In 2013, he stated he intends to direct a Polish novel "Zły" by Leopold Tyrmand. Żuławski and his wife Maria Strzelecka had 2 children together: son Kaj Żuławski (born 2002) and daughter Jagna Żuławska (born 2009).', 'Snow White and the Seven Dwarfs (1955 film)': 'Snow White and the Seven Dwarfs( USA:" Snow White") is a 1955 German film, directed by Erich Kobler, based on the story of Schneewittchen by the Brothers Grimm.', 'Maheen Khan': 'Maheen Kha

In [14]:
# Get the passages and their titles from our processed data
passages_map = processed_data['passages']
corpus_titles = list(passages_map.keys())
print(corpus_titles[:20])  # Print first 5 titles for sanity check
corpus_texts = list(passages_map.values())
print(corpus_texts[:20])

# Tokenize the corpus (simple split is fine for BM25)
tokenized_corpus = [doc.split(" ") for doc in corpus_texts]

# Create the BM25 model and index the corpus
bm25 = BM25Okapi(tokenized_corpus)

# Tokenize the query
question = processed_data['question']
tokenized_query = question.split(" ")

# Get scores for all passages
doc_scores = bm25.get_scores(tokenized_query)

# Rank the titles based on the scores
# We pair titles with scores, sort descending, then extract the titles
ranked_results = sorted(zip(corpus_titles, doc_scores), key=lambda x: x[1], reverse=True)
ranked_titles = [title for title, score in ranked_results]

# Print the top 5 results
print("--- BM25 Top 5 Ranked Titles ---")
pprint(ranked_titles[:5])

['Xawery Żuławski', 'Snow White and the Seven Dwarfs (1955 film)', 'Maheen Khan', 'A Snow White Christmas', 'Alice Washburn', 'Polish-Russian War (film)', 'Viktor Yeliseyev', 'Minamoto no Chikako', 'Liberty Ross', 'Snow White and the Three Stooges']
['Xawery Żuławski (born 22 December 1971 in Warsaw) is a Polish film director. In 1995 he graduated National Film School in Łódź. He is the son of actress Małgorzata Braunek and director Andrzej Żuławski. His second feature "Wojna polsko-ruska" (2009), adapted from the controversial best-selling novel by Dorota Masłowska, won First Prize in the New Polish Films competition at the 9th Era New Horizons Film Festival in Wrocław. In 2013, he stated he intends to direct a Polish novel "Zły" by Leopold Tyrmand. Żuławski and his wife Maria Strzelecka had 2 children together: son Kaj Żuławski (born 2002) and daughter Jagna Żuławska (born 2009).', 'Snow White and the Seven Dwarfs( USA:" Snow White") is a 1955 German film, directed by Erich Kobler, b